In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import yaml 
import numpy as np
import folium
from pyproj import Proj, transform
from h3 import h3
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
paradas.head()

In [ ]:
def visualizar_tramo_folium(tramos,id_tramo):

    tramo1 = tramos.loc[tramos.id_tramo==id_tramo,:].iloc[0]

    #si no hay origen siguiente, tomo el primero del dia 
    if id_tramo == tramos.id_tramo.iloc[-1]:
        origen_siguiente = tramos.id_tramo.iloc[0]
    else:
        origen_siguiente = id_tramo+1
        
    tramo2 = tramos.loc[tramos.id_tramo==origen_siguiente,:].iloc[0]
    
    
    if tramo1.d_h3 is None:
        pass
    else:
        tramo1.lat_d = h3.h3_to_geo(tramo1.d_h3)[0]
        tramo1.lon_d = h3.h3_to_geo(tramo1.d_h3)[1]
        
    #linea_trx = tramo1.linea
    
    #if linea_trx in equivalencias.keys():
    #    linea_carto = equivalencias[linea_trx]
    #else:
    #    linea_carto = linea_trx
    ramal_trx = tramo1.id_ramal
    linea_trx = paradas.loc[paradas.id_ramal == ramal_trx,'id_linea'].unique()
    paradas_destino_posibles = paradas.loc[paradas.id_linea.isin(linea_trx),['lat','lon']]
    #paradas_destino_posibles = paradas.loc[paradas.LINEA == linea_carto,['LATITUDE','LONGITUDE']]
    
    if tramo1.d_h3 is None:
        miny = min(tramo1.o_lat, tramo2.o_lat) - 0.06
        maxy = max(tramo1.o_lat, tramo2.o_lat) + 0.06
        minx = min(tramo1.o_lon, tramo2.o_lon) - 0.06
        maxx = max(tramo1.o_lon, tramo2.o_lon) + 0.06
    else:
        miny = min(tramo1.o_lat, tramo2.o_lat) - 0.06
        maxy = max(tramo1.o_lat, tramo2.o_lat) + 0.06
        minx = min(tramo1.o_lon, tramo2.o_lon) - 0.06
        maxx = max(tramo1.o_lon, tramo2.o_lon) + 0.06


    x_centro,y_centro = minx + ((maxx - minx)/2),miny + ((maxy - miny)/2)

    m = folium.Map(
        location=[y_centro,x_centro],
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    for i in paradas_destino_posibles.index:
        folium.CircleMarker(
            radius=1,
            location=[paradas_destino_posibles.lat.loc[i],paradas_destino_posibles.lon.loc[i]],
            color='black',
            fill=True,
            fill_color='black'
        ).add_to(m)
        
        
    folium.Circle(
        radius=100,
        location=[tramo1.o_lat,tramo1.o_lon],
        popup='Origen '+str(tramo1.hora),
        color='crimson',
        fill=False,
    ).add_to(m)
    
    if tramo1.d_h3 is None:
        pass
    else:

        folium.Marker(
            location=[tramo1.lat_d,tramo1.lon_d],
            popup='Destino',
            icon=folium.Icon(icon='glyphicon-remove-circle',color='red')
        ).add_to(m)
    
    
    folium.Circle(
        radius=100,
        location=[tramo2.o_lat,tramo2.o_lon],
        popup='Origen 2 '+str(tramo2.hora),
        color='grey',
        fill=False,
    ).add_to(m)

                  
    return m

In [ ]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [ ]:
resolucion = 10

In [ ]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [ ]:
q = """
select *
from paradas p
"""
paradas = pd.read_sql(q, conn)

In [ ]:
paradas.head()

In [ ]:
#traer tramos de 10 tarjetas que hayan usado la linea b
query = """
select *
from tramos
where id_tarjeta in (37030200,
  37030272,
  37030633,
  37032074,
  37032172,
  37032419,
  37032508,
  37033376,
  37034039,
  37034278)
order by id_tarjeta,id_viaje,id_tramo;
"""
tramos = pd.read_sql(query, conn)
tramos.head()

In [ ]:
tramo = tramos.loc[tramos.id_tarjeta == 37034039]
tramo

In [ ]:
visualizar_tramo_folium(tramos = tramo,id_tramo = 1)

In [ ]:
def visualizar_tramo_folium(tramos,tramo_id):

    tramo1 = tramos.loc[tramos.tramo_id==tramo_id,:].iloc[0]

    #si no hay origen siguiente, tomo el primero del dia 
    if tramo_id == tramos.tramo_id.iloc[-1]:
        origen_siguiente = tramos.tramo_id.iloc[0]
    else:
        origen_siguiente = tramo_id+1
        
    tramo2 = tramos.loc[tramos.tramo_id==origen_siguiente,:].iloc[0]
    
    
    if tramo1.h3_d is None:
        pass
    else:
        tramo1.lat_d = h3.h3_to_geo(tramo1.h3_d)[0]
        tramo1.lon_d = h3.h3_to_geo(tramo1.h3_d)[1]
        
    linea_trx = tramo1.linea
    
    if linea_trx in equivalencias.keys():
        linea_carto = equivalencias[linea_trx]
    else:
        linea_carto = linea_trx
    
    paradas_destino_posibles = paradas.loc[paradas.LINEA == linea_carto,['LATITUDE','LONGITUDE']]
    if tramo1.h3_d is None:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06
    else:
        miny = min(tramo1.lat_o, tramo2.lat_o) - 0.06
        maxy = max(tramo1.lat_o, tramo2.lat_o) + 0.06
        minx = min(tramo1.lon_o, tramo2.lon_o) - 0.06
        maxx = max(tramo1.lon_o, tramo2.lon_o) + 0.06


    x_centro,y_centro = minx + ((maxx - minx)/2),miny + ((maxy - miny)/2)

    m = folium.Map(
        location=[y_centro,x_centro],
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    for i in paradas_destino_posibles.index:
        folium.CircleMarker(
            radius=1,
            location=[paradas_destino_posibles.LATITUDE.loc[i],paradas_destino_posibles.LONGITUDE.loc[i]],
            color='black',
            fill=True,
            fill_color='black'
        ).add_to(m)
        
        
    folium.Circle(
        radius=100,
        location=[tramo1.lat_o,tramo1.lon_o],
        popup='Origen '+str(tramo1.fecha).split(' ')[1],
        color='crimson',
        fill=False,
    ).add_to(m)
    
    if tramo1.h3_d is None:
        pass
    else:

        folium.Marker(
            location=[tramo1.lat_d,tramo1.lon_d],
            popup='Destino',
            icon=folium.Icon(icon='glyphicon-remove-circle',color='red')
        ).add_to(m)
    
    
    folium.Circle(
        radius=100,
        location=[tramo2.lat_o,tramo2.lon_o],
        popup='Origen 2 '+str(tramo2.fecha).split(' ')[1],
        color='grey',
        fill=False,
    ).add_to(m)

                  
    return m

In [ ]:
with open('../recorridos_equivalencias.yaml') as file:
    equivalencias = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
visualizar_tramo_folium(tramos,5)

In [ ]:
q = """
select p."LINEA", p."LATITUDE",p."LONGITUDE",p."h3_res_%i"
from paradas p
"""%resolucion
paradas = pd.read_sql(q, conn)

In [ ]:
# traer trx de tarjetas que hayan pasado por la B SIN MISSING
q = """
select distinct tarjeta
FROM tramos_linea_b
where h3_d is not null
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [ ]:
# traer trx de tarjetas que hayan pasado por la B CON MISSING
q = """
select distinct tarjeta
FROM tramos_linea_b
where h3_d is null
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [ ]:
# traer trx de tarjetas que hayan pasado por la B 
q = """
select distinct tarjeta
FROM tramos_linea_b
"""
tarjetas_con_destino = pd.read_sql(q, conn)

In [ ]:
#1263039 combina tren bus y subte
#1233654 tipico pendular
#1248216 ejemplo de perder un caso
#18082 ejemplo de perder un caso porque camina mucho (actividad mediante quizas)
#9099909 ejemplo de tarjeta que no respeta el ultimo viaje con destino 
tarjeta = tarjetas_con_destino.sample(1).iloc[0,0]

In [ ]:
tarjeta = 147

In [ ]:
q = """
select *
FROM tramos_linea_b
where tarjeta = %i
order by tramo_id
"""%tarjeta
tramos = pd.read_sql(q, conn)
tramos.sort_values('tramo_id').iloc[:,1:]

In [ ]:
visualizar_tramo_folium(tramos,5)